

На сайте проведён AB тест.  
Нужно проанализировать конверсию, полученую в следствии изменения. Показатель, на который мы будем смотреть - это количество кликов по кнопке "Изучить курсы" на домашней странице. 
Показатель кликабельности определяется как количество кликов, деленное на количество просмотров.

Измеряемая метрика:   
CR: # клики уникальными пользователями / # просмотры уникальными пользователями

Нулевая гипотеза:   
Изменение конверсии не произошло.
H0: CR_espirement ≤ CR_control

Альтернативная гипотеза:  
Данные контрольной и экспериментальной группы статистически различны.
H1: CR_espirement ≤ CR_control

Альтернативная гипотеза - это то, что мы хотим доказать, в данном случае, что новый дизайн домашней страницы имеет более высокий показатель кликов, чем старый дизайн домашней страницы.

In [1]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/audacity-ab-testing/homepage_actions.csv


Посмотрим как выглядят данные:

In [2]:
df = pd.read_csv('/kaggle/input/audacity-ab-testing/homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


Проведём некоторые преобразования.

In [3]:
df = df.rename(columns={'id':'user_id'})            # переименуем столбец для удобства
df['timestamp'] = pd.to_datetime(df['timestamp'])   # преобразуем формат для даты
df['timestamp'].max() - df['timestamp'].min()       # найдём количество дней от начальной до конечной даты

Timedelta('115 days 16:41:40.789831')

Эксперимент длился 115 дней.

Для чистоты теста проверим уникальность пользователей, user_id не должен повторяться для действия "view".

In [4]:
a = df['user_id'].value_counts().to_frame()    # user_id повторяется максимум 2 раза
b = a.query('count > 1')                       # список user_id, которые встречаются больше 1 раза
rslt = df[df['user_id'].isin(b.index)]
rslt.shape
rslt = rslt.groupby(['group', 'action'])['action'].agg('count')
print(rslt)      # сгрупированная таблица для повторяющихся user_id

group       action
control     click     932
            view      932
experiment  click     928
            view      928
Name: action, dtype: int64


user_id повторяется только в случае если было действие 'click'.   
Значит user_id - уникальны.

Группируем данные по group(группа) и action(действию).

In [5]:
df2 = df.groupby(['group', 'action'])['action'].agg('count')
df2

group       action
control     click      932
            view      3332
experiment  click      928
            view      2996
Name: action, dtype: int64

Из полученной таблицы можно найти конверсию.   
CR = #клики уникальными пользователями / #просмотры уникальными пользователями

In [6]:
# зададим переменные для расчётов
control_view = df.query("action == 'view' and group == 'control'")['action'].count()
control_click = df.query("action == 'click' and group == 'control'")['action'].count()
experiment_view = df.query("action == 'view' and group == 'experiment'")['action'].count()
experiment_click = df.query("action == 'click' and group == 'experiment'")['action'].count()
# посчитаем конверсию для двух групп
cr_control = round(control_click / control_view, 4)
cr_expirement = round(experiment_click / experiment_view, 4)
print("conversion control = ", cr_control, "conversion expirement = ", cr_expirement)
# оценим результат
res = round((cr_expirement* 100 / cr_control) - 100, 1)
print (res, '%')

conversion control =  0.2797 conversion expirement =  0.3097
10.7 %


Конверсия в экспериментальной группе выше на 10,7 %.

Проверим на сколько статистически значемые результаты получились при помощи z-теста.

In [7]:
z, p_value = proportions_ztest([control_click, experiment_click], [control_view, experiment_view])
round(p_value, 5)

0.00883

p_value = 0,00883 меньше 0,05 - отвергаем нулевую гипотезу. Данные контрольной и экспериментальной группы статистически различны.   
Рекумендуется ввести изменения для всех пользователей.